In [ ]:
from google.colab import files
uploaded = files.upload()

Saving FINAL_ESSAYS.csv to FINAL_ESSAYS.csv


In [2]:
# Import Pandas library
import pandas as pd

# Load the first dataset
dataset1_url ="https://github.com/SevithaJanga03/LLM---Detect-AI-Generated-Text/blob/main/final_essays.csv"   # Replace with the actual URL or file path
df = pd.read_csv('final_essays.csv')

# Display dataset
print(df.head())


         id  prompt_id                                               text  \
0  0059830c          0  Cars. Cars have been around since they became ...   
1  005db917          0  Transportation is a large necessity in most co...   
2  008f63e3          0  "America's love affair with it's vehicles seem...   
3    940276          0  How often do you ride in a car? Do you drive a...   
4  00c39458          0  Cars are a wonderful thing. They are perhaps o...   

   generated  
0          0  
1          0  
2          0  
3          0  
4          0  


In [9]:
# Specify the features (X) and target variable (y)
from sklearn.model_selection import train_test_split
X = df
# X = df.drop('generated', axis=1)  # Assuming 'generated' is the target variable
y = df['generated']

# Split the dataset into training and development sets
# Adjust the test_size and random_state parameters as needed
X_train, X_dev, y_train, y_dev = train_test_split(X, y, test_size=0.2, random_state=42)

# Display the shapes of the resulting sets
print("Training set shape:", X_train.shape, y_train.shape)
print("Development set shape:", X_dev.shape, y_dev.shape)

Training set shape: (1126, 4) (1126,)
Development set shape: (282, 4) (282,)


KeyError: ignored

In [48]:
from collections import Counter

my_dict= {}
def build_vocabulary(essay, min_occurrence=5):

    # Flatten the list of sentences into a single list of words
    all_words = [word.lower() for word in essay.split()]

    # Count the occurrences of each word
    word_counts = Counter(all_words)

    # Filter out rare words based on the minimum occurrence threshold
    vocabulary = [word for word, count in word_counts.items() if count >= min_occurrence]

    # Create a reverse index mapping each word to its index in the vocabulary
    reverse_index = {word: index for index, word in enumerate(vocabulary)}
    print(vocabulary)
    return reverse_index

for index,row in X_train.iterrows():
  reverse_index = build_vocabulary(row['text'])
  if(row['generated'] == 0):
    Class_voc={'HUMAN':0}
    Class_voc.update(reverse_index)
  else:
    Class_voc={'LLM':1}
    Class_voc.update(reverse_index)
  try:
    my_dict[row['id']].append(Class_voc)
  except KeyError:
    my_dict = {**my_dict, **{row['id']: Class_voc}}

print(my_dict)

['i', 'not', 'in', 'of', 'the', 'electoral', 'it', 'this', 'you', 'vote', 'for', 'a', 'without', 'have', 'no', 'control', 'over', 'who', 'electors', 'should', 'our', 'and', 'that', 'we', 'is', 'people', 'states', 'be', 'to', 'do']
['you', 'the', 'and', 'of', 'having', 'not', 'a', 'people', 'car', 'some', 'to', 'it', 'or', 'have', 'their', 'on', 'is', 'for', 'with', 'percent', 'was', 'this', 'in', 'driving', 'cars', 'that']
['of', 'the', 'to', 'more', 'cars', 'and', 'it', 'are', 'that', 'would', 'be', 'for', 'car', 'usage', 'in', 'has', 'much', 'around', 'they']
['the', 'world', 'people', 'are', 'it', 'be', 'or', 'they', 'to', 'an', 'of', 'cars', 'on', 'a', 'in', 'there', 'is', 'city', 'and', 'this', 'with', 'pollution', 'that', 'would', 'paris', 'smog', 'air', 'driving', 'was', 'have', 'their', 'cities', 'by', 'day', 'from', 'most', 'not', 'completely', 'other', 'amount', 'according', 'article', 'paragragh', 'into', 'car', 'more']
['the', 'electoral', 'college', 'for', 'many', 'it', 'i

In [44]:
def calculate_probability(word, documents):
    num_documents_with_word = sum(1 for document in documents.values() if word in document)
    # Calculate the probability
    probability = num_documents_with_word / len(X_train)
    return probability

def calculate_conditional_probability(word, documents,class_label):
    num_documents_in_class=0
    num_positive_documents=0
    for document in documents.values():
      if(word in document and list(document.items())[0][0]==class_label):
        num_positive_documents=num_positive_documents+1
      if(list(document.items())[0][0]==class_label):
        num_documents_in_class=num_documents_in_class+1
    probability = num_positive_documents / num_documents_in_class
    return probability


# Word for which probability is calculated
target_word = "the"
class_label = "HUMAN"
# Calculate the probability of the word
probability = calculate_probability(target_word, my_dict)
llm_conditional_probability = calculate_conditional_probability(target_word, my_dict,class_label)

# Display the result
print(f"Probability of '{target_word}': {probability:.4f}")
print(f"Probability of '{target_word} in {class_label} class ': {llm_conditional_probability:.4f}")

Probability of 'the': 0.9902
Probability of 'the in HUMAN class ': 0.9900


In [54]:
X_traindata = X_train[['text','generated']].values.tolist()

def calculate_probabilities(word_counts, class_counts):
    probabilities = {}

    for (word, label), count in word_counts.items():
        probabilities[(word, label)] = count / class_counts[label]

    return probabilities
def count_words(data):
    # Initialize dictionaries to store word counts and class counts
    word_counts = defaultdict(int)
    class_counts = defaultdict(int)

    # Iterate through each entry in the dataset
    for text, label in data:
        # Tokenize the text using the tokenize function
        words = tokenize(text)
        # Update word counts and class counts based on the tokenized words and labels
        for word in words:
            word_counts[(word, label)] += 1
            class_counts[label] += 1

    # Return the dictionaries containing word counts and class counts
    return word_counts, class_counts
words_counts,class_counts = (count_words(X_traindata))
print('Words counts : ',words_counts)
print('Class Counts : ',class_counts)
class_probabilities = {label: count / len(X_traindata) for label, count in class_counts.items()}
print(class_probabilities)
probabilities = calculate_probabilities(word_counts, class_counts)
print(probabilities)


defaultdict(<class 'int'>, {('dear', 0): 217, ('senator', 0): 310, (',', 0): 26062, ('i', 0): 2211, ('am', 0): 117, ('not', 0): 4658, ('in', 0): 13196, ('favor', 0): 199, ('of', 0): 17741, ('the', 0): 38996, ('electoral', 0): 5686, ('college', 0): 4746, ('.', 0): 27961, ('may', 0): 913, ('seem', 0): 182, ('helpful', 0): 43, ('some', 0): 1312, ('ways', 0): 318, ('terms', 0): 18, ('voting', 0): 1372, ('but', 0): 2702, ('entirely', 0): 46, ('think', 0): 723, ('it', 0): 7233, ('this', 0): 4739, ('way', 0): 1393, ('you', 0): 3746, ('vote', 0): 4691, ('for', 0): 7462, ('a', 0): 15077, ('specific', 0): 47, ('candidate', 0): 1223, ('are', 0): 5972, ('without', 0): 883, ('knowing', 0): 89, ('whether', 0): 114, ('or', 0): 2575, ('your', 0): 1370, ('votes', 0): 1962, ('making', 0): 324, ('difference', 0): 99, ('because', 0): 2345, ('have', 0): 5156, ('no', 0): 1217, ('control', 0): 189, ('over', 0): 714, ('who', 0): 2091, ('electors', 0): 1983, ('believe', 0): 443, ('system', 0): 1456, ('should',

In [37]:
import nltk
import random
from collections import defaultdict

# Download NLTK data
nltk.download('punkt')

def tokenize(text):
    return nltk.word_tokenize(text.lower())

def split_data(data, split_ratio=0.8):
    random.shuffle(data)
    split_idx = int(len(data) * split_ratio)
    train_data = data[:split_idx]
    dev_data = data[split_idx:]
    return train_data, dev_data

def count_words(data):
    word_counts = defaultdict(int)
    class_counts = defaultdict(int)

    for text, label in data:
        words = tokenize(text)
        for word in words:
            word_counts[(word, label)] += 1
            class_counts[label] += 1

    return word_counts, class_counts

def calculate_probabilities(word_counts, class_counts):
    probabilities = {}

    for (word, label), count in word_counts.items():
        probabilities[(word, label)] = count / class_counts[label]

    return probabilities

def predict_class(text, probabilities, class_probabilities):
    words = tokenize(text)
    log_prob_human = 0
    log_prob_llm = 0

    for word in words:
        log_prob_human += probabilities.get((word, 0), 0)
        log_prob_llm += probabilities.get((word, 1), 0)

    log_prob_human += class_probabilities[0]
    log_prob_llm += class_probabilities[1]

    return 0 if log_prob_human > log_prob_llm else 1

def evaluate_accuracy(dev_data, probabilities, class_probabilities):
    correct_predictions = 0

    for text, true_label in dev_data:
        predicted_label = predict_class(text, probabilities, class_probabilities)
        if predicted_label == true_label:
            correct_predictions += 1

    accuracy = correct_predictions / len(dev_data)
    return accuracy


data1 = df[['text','generated']]
data = data1.values.tolist()
print(data)

train_data, dev_data = split_data(data)

word_counts, class_counts = count_words(train_data)
class_probabilities = {label: count / len(train_data) for label, count in class_counts.items()}
probabilities = calculate_probabilities(word_counts, class_counts)

accuracy = evaluate_accuracy(dev_data, probabilities, class_probabilities)
print(f"Accuracy on dev data: {accuracy}")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Accuracy on dev data: 0.975177304964539


In [28]:
import re
from collections import defaultdict


my_pred=[]
class NaiveBayesClassifier:
    def __init__(self):
        self.class_probabilities = defaultdict(int)
        self.word_given_class_probabilities = defaultdict(lambda: defaultdict(int))
        self.vocabulary = set()

    def preprocess_text(self, text):
        # Simple text preprocessing: lowercase and remove non-alphanumeric characters
        text = text.lower()
        text = re.sub(r'[^a-z0-9\s]', '', text)
        return text

    def train(self, documents, labels):
        # Calculate class probabilities
        total_documents = len(documents)
        for label in set(labels):
            documents_in_class = [doc for doc, doc_label in zip(documents, labels) if doc_label == label]
            self.class_probabilities[label] = len(documents_in_class) / total_documents

            # Process and update word counts for each document in the class
            for document in documents_in_class:
                document = self.preprocess_text(document)
                for word in document.split():
                    self.word_given_class_probabilities[label][word] += 1
                    self.vocabulary.add(word)

    def predict(self, document):
        document = self.preprocess_text(document)
        scores = defaultdict(float)
        print(self.class_probabilities)
        print(self.vocabulary)
        # Calculate scores for each class
        for label in self.class_probabilities:
            scores[label] = 0.0
            for word in document.split():
                # Apply Laplace smoothing
                scores[label] += (
                    (self.word_given_class_probabilities[label][word] + 1) /
                    (len(self.vocabulary) + len(self.word_given_class_probabilities[label]))
                )

            scores[label] += self.class_probabilities[label]

        # Return the label with the highest score
        return max(scores, key=scores.get)

documents = X_train['text']
labels = y_train



classifier = NaiveBayesClassifier()
classifier.train(documents, labels)

for index,row in X_dev.iterrows():
  new_document = row['text']
  prediction = classifier.predict(new_document)
  my_pred.append(prediction)

print(f"Predictions: {my_pred}")


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [20]:
import re
from collections import defaultdict

class NaiveBayesClassifier:
    def __init__(self):
        self.class_probabilities = defaultdict(int)
        self.word_given_class_probabilities = defaultdict(lambda: defaultdict(int))
        self.vocabulary = set()

    def preprocess_text(self, text):
        # Simple text preprocessing: lowercase and remove non-alphanumeric characters
        text = text.lower()
        text = re.sub(r'[^a-z0-9\s]', '', text)
        return text

    def train(self, documents, labels):
        # Calculate class probabilities
        total_documents = len(documents)
        for label in set(labels):
            documents_in_class = [doc for doc, doc_label in zip(documents, labels) if doc_label == label]
            self.class_probabilities[label] = len(documents_in_class) / total_documents

            # Process and update word counts for each document in the class
            for document in documents_in_class:
                document = self.preprocess_text(document)
                for word in document.split():
                    self.word_given_class_probabilities[label][word] += 1
                    self.vocabulary.add(word)

    def predict(self, document):
        document = self.preprocess_text(document)
        scores = defaultdict(float)

        # Calculate scores for each class
        for label in self.class_probabilities:
            scores[label] = 0.0
            for word in document.split():
                # Apply Laplace smoothing
                scores[label] += (
                    (self.word_given_class_probabilities[label][word] + 1) /
                    (len(self.vocabulary) + len(self.word_given_class_probabilities[label]))
                )

            scores[label] += self.class_probabilities[label]

        # Return the label with the highest score
        return max(scores, key=scores.get)


# Example Usage:
documents = [
    "The cyber age has ushered in a new era of possibilities and challenges for democratic institutions, and the Electoral College stands at the forefront of this transformation. This essay explores the intersection of the Electoral College and the cyber age, examining how the system adapts to the evolving dynamics of digital communication, cybersecurity, and the reshaping of political landscapes. Proponents argue that the Electoral College remains essential in preserving state representation and preventing the undue influence of major population centers. By requiring candidates to engage with a diverse array of states, the system aims to maintain a balanced and representative democracy. Supporters also contend that the Electoral College contributes to political stability by discouraging the rise of numerous candidates and maintaining a two-party system, considerations that take on added significance in the context of cybersecurity challenges. However, the cyber age introduces new complexities and vulnerabilities for the Electoral College. Concerns about election interference, disinformation campaigns, and cyber threats underscore the importance of securing the electoral process. Instances where the winner of the popular vote did not secure the presidency, as observed in the elections of 1876, 1888, and 2000, become heightened concerns in an era where safeguarding the integrity of elections is paramount. As citizens navigate the digital frontiers of democracy, the conversation about the Electoral College extends beyond its traditional roles to address fundamental questions of election security and adaptability in the cyber age. The ongoing dialogue prompts reflection on how the Electoral College can reinvent itself to meet the challenges and opportunities presented by the digital era while upholding the core tenets of representation, fairness, and the democratic will of the people.",
    "Automobiles are one of the most dramatic and resourceful products of the Industrial Revolution. They help people commute from one place to another, allow for travel time to decrease, and help citizens carry heavy objects from point A to point B. Only, did we think about the negative effect it could have on our environment? Forests are cut down to build highways, accidents are caused when the driver fails to pay proper attention to what he or she is doing, and greenhouse gases have overridden our atmosphere with choking smog. Many Americans rely on their cars, but have we thought about the positive of limiting our car usage? We as fellow citizens have a substantial amount of advantages if we simply limit our car usage to help create a better environment, for us, for the community, and for those to come. One advantage of limited car usage is a better quality of life for a person. When a person has the responsibility of a car, they must make sure all of the parts are working properly, know how to operate the car safely, know exactly which routes to take that are safest for the car and for themself, know exactly how to fix the car in case something malfunctions, the list continues. This causes the person to become overwhelmed and stressed due to the needs of their car. Elisabeth Rosenthal states in ""The End of Car Culture"" that, ""people who stopped car commuting as a result of recession find little reason to resume the habit"" Rosenthal, paragraph 8. Basically, people who limited their use of cars were more relaxed and found no good reason to return to their habit of using a car. furthermore, with more people limiting their car usage, the more they use local stores and shops that are walking or biking distance. Rosenthal also states ""In German Suburb, Life Goes on Without Cars"" , ""stores are placed a walk away, on a main street, rather than in malls..."" Rosenthal, paragraph 6. In other words, businesses florish due to their locations and promote citizens to limit their car usage due to their distance to travel. This also allows for more people to acknowledge their cities and allow them to become cleaner and beautiful. Similar to the first, another advantage is for the betterment of the city. According to ""Paris bans driving due to smog"" by Robert Duffer, ""... Beijing, China which is known as one of the most polluted cities in the world"" Duffer, paragraph 5. This is due to Beijing's huge population tourist and regular and their increased use of cars on a daily basis. In other words, smog due to passenger cars in one of the leading causes of pollution in our society. The cars are, literally, choking us and future generation to the point where we won't be able to breathe. Limited car usage allows for the reduction of greenhouse emmisions and smog, allowing for oxygen to circulate and breathe life into us and our children. As a result of reduced greenhouse emmisions, cities become cleaner and more beautiful. As stated by Andrew Selsky in ""Carfree day is Spinning into a Big Hit in Bogota"" , ""Parks and sports senters also have bloomed throughout the city uneven, pitted sidewalks have been replaced by broad, smooth sidewalks and new resturants and upscale shopping districts have croopped up"" Selsky, paragraph 15. Basically, the limited use of cars has allowed the city to florish and reconstruct itself into something beautiful and strong for the community to enjoy. The limited use of cars has allowed for the betterment of the community and of cities across the globe and many people are taking advantage of this new era coming into play. The health of people has improved and the pollution that makes up our atmosphere has decreased in a slow, but steady stream. Imagine how much better the environment would be once the use of cars is limited to less than one perent of the population. The future is bright and we only need to strive for it, without headlights.",
    "All across the globe, a revolutionary idea is being set in motion. The idea of essentially banning cars from the streets has sparked as much interest as it has critics. The automotive industry has been booming since World War 2, and although the positive effects of cars can be seen in our daily lives, we seldom take into consideration of the negative impacts of cars. Effects such as pollution, traffic, and cost have impacted our lives as automotive users, even if we have not realized it. Pollution is one of the primary concerns of a world that is constantly driving, yet it is something people hardly take into account when driving. In theory that is understandable, considering the last thing a person is thinking about when they're driving is what their car is spitting out into the atmosphere. However pollution doesn't go unnoticed for long. For example, the city of Paris, France enforced a partial driving ban in order to decrease the almost record breaking pollution across the city, stating, ""The smog rivaled Beijing, China, which is known as one of the most polluted cities in the world."" The smog did clear from the city and previous driving laws were reinstated, however this shows how car pollution has a negative impact on the world around us. Another negative impact of driving is traffic. Traffic is a driver's worst nightmare, and it affects all of us at some point in our lives. In Bogota, Columbia, citizens participated in a drivingfree day, where everyone was encouraged to either hike, bike, skate, or use public transportation in order to reach their destination. This event, in which millions of Colombians participated, was reportedly ""leaving the streets of this capital city eerily devoid of traffic jams."" This program left a positive mark on the city of Bogota, and will hopefully soon spread to other countries, reducing crowded, traffic filled streets around the world. The world runs on money, just ask any Wall Street broker. This is a paradox concerning both car owners and people looking to purchase a car. The question remains, if we're all trying to save money, why spend thousands of dollars on a costly car? According to a census taken in 2013, ""Part of the explanation certainly lies in the recession, because cashstrapped Americans could not afford cars, and the unemployed weren't going to work anyway."" This shows that the number of people buying cars is steadily decreasing. Another costly problem of cars is fuel. Buying gasoline for your car every week can get incredibly costly, especially when driving a large truck or sports car. This shows how the cost of cars can have a negative impact on our lives. In conclusion, the use of cars is steadily declining. Cars just aren't a very positive motive for transportation anymore considering the effects of pollution, traffic, and cost that cars produce.",
    "In a world where the automobile has long been synonymous with urban living, the concept of car-free cities represents a bold paradigm shift. This essay delves into the multifaceted aspects of this transformative idea, exploring the motivations, challenges, and potential benefits of reimagining urban landscapes without private cars. The environmental imperative is a primary driver behind the push for car-free cities. Traditional transportation models, heavily reliant on internal combustion engine vehicles, contribute significantly to air pollution and greenhouse gas emissions. By embracing alternative modes of transportation and limiting or eliminating private cars, cities can curb their environmental impact, mitigating climate change and creating healthier environments for residents. Car-free cities prioritize people over vehicles, reshaping urban spaces into pedestrian-friendly zones. This shift fosters community engagement, encourages social interactions, and enhances overall well-being. The emphasis on walkable streets, green spaces, and human-centric design transforms the urban landscape into a space that prioritizes the needs of its inhabitants, promoting a higher quality of life. Challenges in implementing car-free cities are undeniable, ranging from public resistance to the need for significant infrastructure changes. However, successful case studies illustrate that these challenges are not insurmountable. Cities that have successfully navigated this paradigm shift provide valuable insights into overcoming obstacles, fostering community engagement, and reaping the long-term benefits of sustainable urban planning."
]

labels = [0, 1, 0, 0]

classifier = NaiveBayesClassifier()
classifier.train(documents, labels)

new_document = "The transition to car-free living heralds a cultural renaissance in urban environments, reshaping the way communities interact, engage, and define their shared identity. This essay delves into how the absence of private cars contributes to the creation of vibrant, dynamic, and culturally rich cities. Central to this cultural renaissance is the reclaiming of urban spaces. The reduction or elimination of private cars transforms streets, once dominated by vehicular traffic, into communal areas that encourage social interaction and cultural expression. Parks, squares, and public spaces become stages for artistic performances, community events, and cultural celebrations, fostering a sense of collective identity and pride. Car-free living also promotes alternative modes of transportation that inherently encourage exploration and discovery. Walkable streets, cycling paths, and efficient public transit systems create opportunities for residents to engage with their surroundings on a more intimate level. The vibrant street life becomes a canvas for diverse cultural experiences, from street art to local markets, creating an urban tapestry that reflects the richness of the community. Furthermore, the emphasis on inclusivity in car-free cities ensures that cultural experiences are accessible to all residents. The reduced dominance of cars creates environments where people of all ages, backgrounds, and abilities can participate in the cultural life of the city. This inclusivity fosters a sense of cultural democracy, where everyone has the opportunity to contribute to and benefit from the collective cultural wealth of the community. In conclusion, the cultural renaissance spurred by car-free living goes beyond transportation; it redefines the very essence of urban living. As cities become cultural hubs, shaped by the people who inhabit them, the vision of car-free living emerges as a catalyst for a new era of cultural vibrancy, diversity, and community engagementâ€”a renaissance that positions cities as dynamic and evolving expressions of the human experience."
prediction = classifier.predict(new_document)

print(f"Prediction: {prediction}")


Prediction: 0


In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.classify import NaiveBayesClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

nltk.download('punkt')

# Sample dataset
documents = [
    ("This is a human-written essay.", 'human'),
    ("The language model generated this text.", 'ai'),
    ("Humans can express complex ideas.", 'human'),
    ("Language models rely on pre-existing patterns.", 'ai'),
    # Add more data as needed
]

# Tokenization and preprocessing
def preprocess_text(text):
    data = text.lower()
    words = word_tokenize(data)
    return FreqDist(words)

# Feature extraction
features = [(preprocess_text(text), label) for (text, label) in documents]

# Split the dataset into train and development sets
train_set, dev_set = train_test_split(features, test_size=0.2, random_state=42)

# Train the Naive Bayes Classifier
classifier = NaiveBayesClassifier.train(train_set)

# Calculate vocabulary and reverse index
all_words = [word for (freq_dist, label) in train_set for word in freq_dist.keys()]
vocabulary = [word for word, count in FreqDist(all_words).items() if count >= 5]
reverse_index = {word: index for index, word in enumerate(vocabulary)}

# Calculate probabilities
def calculate_occurrence_probability(word, dataset):
    num_documents_with_word = sum(1 for (freq_dist, label) in dataset if word in freq_dist)
    probability = num_documents_with_word / len(dataset)
    return probability

def calculate_conditional_probability(word, label, dataset):
    num_positive_documents = sum(1 for (freq_dist, doc_label) in dataset if doc_label == label and word in freq_dist)
    num_documents_in_class = sum(1 for (_, doc_label) in dataset if doc_label == label)
    probability = num_positive_documents / num_documents_in_class
    return probability

# Example usage of probabilities
target_word = 'the'
occurrence_probability = calculate_occurrence_probability(target_word, train_set)
llm_conditional_probability = calculate_conditional_probability(target_word, 'ai', train_set)

print(f"Probability of occurrence P['{target_word}']: {occurrence_probability:.4f}")
print(f"Conditional probability P['{target_word}' | LLM]: {llm_conditional_probability:.4f}")

# Calculate accuracy using dev dataset
dev_features = [(preprocess_text(text), label) for (text, label) in dev_set]
dev_true_labels = [label for (_, label) in dev_set]
dev_predicted_labels = [classifier.classify(features) for (features, _) in dev_features]

dev_accuracy = accuracy_score(dev_true_labels, dev_predicted_labels)
print(f"Accuracy on Development Set: {dev_accuracy:.2%}")

# Experiments with Smoothing and Top 10 words
# ... (You can experiment with different smoothing techniques and identify top words)

# Calculate accuracy using the test dataset
# ... (Use the optimal hyperparameters found in the experiments)

# Note: Adjust the code based on the specifics of your dataset and requirements.


Probability of occurrence P['the']: 0.0000
Conditional probability P['the' | LLM]: 0.0000


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


AttributeError: ignored

In [24]:
data1 = df[['text','generated']]
print(data1)

                                                   text  generated
0     Cars. Cars have been around since they became ...          0
1     Transportation is a large necessity in most co...          0
2     "America's love affair with it's vehicles seem...          0
3     How often do you ride in a car? Do you drive a...          0
4     Cars are a wonderful thing. They are perhaps o...          0
...                                                 ...        ...
1403  The advent of social media has revolutionized ...          1
1404  As we step into the cyber era, the Electoral C...          1
1405  The cyber age has ushered in a new era of poss...          1
1406  In the cyber age, the Electoral College confro...          1
1407  In the cyber age, the Electoral College faces ...          1

[1408 rows x 2 columns]
